In [1]:
import numpy as np
import pandas as pd

In [2]:

job_df = pd.read_csv("Combined_Jobs_Final.csv")

In [3]:

job_df.head(2)

,Job.ID,Provider,Status,Slug,Title,Position,Company,City,State.Name,State.Code,...,Industry,Job.Description,Requirements,Salary,Listing.Start,Listing.End,Employment.Type,Education.Required,Created.At,Updated.At
0,111,1,open,palo-alto-ca-tacolicious-server,Server @ Tacolicious,Server,Tacolicious,Palo Alto,California,CA,...,Food and Beverages,Tacolicious' first Palo Alto store just opened...,NaN,8.0,NaN,NaN,Part-Time,NaN,2013-03-12 02:08:28 UTC,2014-08-16 15:35:36 UTC
1,113,1,open,san-francisco-ca-claude-lane-kitchen-staff-chef,Kitchen Staff/Chef @ Claude Lane,Kitchen Staff/Chef,Claude Lane,San Francisco,California,CA,...,Food and Beverages,\r\n\r\nNew French Brasserie in S.F. Financia...,NaN,0.0,NaN,NaN,Part-Time,NaN,2013-04-12 08:36:36 UTC,2014-08-16 15:35:36 UTC


In [4]:
print(len(job_df))
job_df = job_df[['Status', 'Title', 'Position', 'Company', 'Job.Description']]
print(len(job_df))

84090
84090


In [5]:

job_df.shape

(84090, 5)

In [6]:
job_df['Job.Description'][20]

'Hiring Event Details\r\nStore Associate\r\n\r\n$12.00 / Hour\r\nAdditional $1.00 Per Hour For ALL Sunday Shifts!\r\n50 Cent Wage Increases Beginning At 6 Months - Up to $13.50 At 2 Years\r\n\r\nMonday, December 15, 2014\r\n9am - 11am\r\n\r\nALDI\r\n3133 Market Place Dr\r\nOnalaska, WI 54650\r\n\r\n&nbsp;\r\nFor consideration, please apply in person at the hiring event only. Get started now by downloading our Store Employment Application.\r\n\r\nStore Associate - Retail Sales ( Customer Service )\r\n\r\nIf you are a customer service minded individual with a positive and energetic personality and you&rsquo;re interested in working for one of the best-known grocery stores in the nation, join the ALDI family! We are looking for motivated and reliable individuals to serve as a Store Associate. You will serve as the face of ALDI, providing customers with friendly and efficient check-out services. But that&rsquo;s just the beginning. You will also assist the store manager in a variety of rol

In [7]:
job_df.isnull().sum()
job_df.fillna('',inplace=True)
job_df.isnull().sum()

Status             0
Title              0
Position           0
Company            0
Job.Description    0
dtype: int64

In [8]:
job_df = job_df.sample(n=1000,random_state=42)

In [9]:

job_df.shape

(1000, 5)

In [24]:

from nltk.corpus import stopwords
import nltk
import re
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [34]:
import re
import nltk
from nltk.stem import PorterStemmer

def cleaning(txt):
    # Step 1: Clean the text
    cleaned_txt = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    
    # Step 2: Tokenize the cleaned text
    tokens = nltk.word_tokenize(cleaned_txt.lower())
    
    # Step 3: Apply stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # Step 4: Convert the list of stemmed tokens to a string
    stemmed_text = ' '.join(stemmed_tokens)
    
    return stemmed_text


i am clean the text with stem


In [35]:
cleaning("this is my book with 100 digits and \n\n")

'thi is my book with 100 digit and'

In [36]:
job_df['Job.Description'] = job_df['Job.Description'].astype(str).apply(lambda x: cleaning(x))
job_df['Title'] = job_df['Title'].astype(str).apply(lambda x: cleaning(x))
job_df['Position'] = job_df['Position'].astype(str).apply(lambda x: cleaning(x))

In [37]:
job_df['Title']

64119                       site director knowledg univers
35827                          administr assist officeteam
72100                     account manag chi payment system
46355    outsid wholesal sale rep parttim river front c...
34166    custom servic rep help peopl with hear loss ca...
                               ...                        
66282    sale repres sale associ entri level vector market
39515                             staff account accountemp
69231                    unarm secur offic us secur associ
69618              line cook crown plaza independ own oper
6144     kitchen manag job detail the artesian hotel an...
Name: Title, Length: 1000, dtype: object

In [38]:
job_df['Job.Description']

64119    job summari knowledg univers ku site director ...
35827    ref id 03110118480classif secretaryadmin asstc...
72100    if yoursquor energet motiv hardwork and look f...
46355    outsid wholesal sale rep job respons sale repr...
34166    captioncal commun assist imagin life without t...
                               ...                        
66282    nbsp if you are eager to learn we have an oppo...
39515    ref id02120107460classificationaccount staffco...
69231    under direct supervis the unarm secur offic is...
69618    summari respons for maintain and set up food p...
6144     posit descript provid overal leadership superv...
Name: Job.Description, Length: 1000, dtype: object

In [39]:

job_df['clean_text'] = job_df['Job.Description']+" "+job_df['Title']+job_df['Position']

In [40]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
tfidf = TfidfVectorizer(stop_words='english')
matrix = tfidf.fit_transform(job_df['clean_text'])
similarity = cosine_similarity(matrix)

In [42]:
similarity

array([[1.        , 0.04317507, 0.02131549, ..., 0.05101442, 0.02491746,
        0.08041478],
       [0.04317507, 1.        , 0.02933778, ..., 0.03674247, 0.00660794,
        0.02653012],
       [0.02131549, 0.02933778, 1.        , ..., 0.05498327, 0.0295402 ,
        0.04217164],
       ...,
       [0.05101442, 0.03674247, 0.05498327, ..., 1.        , 0.05932786,
        0.11136057],
       [0.02491746, 0.00660794, 0.0295402 , ..., 0.05932786, 1.        ,
        0.38562581],
       [0.08041478, 0.02653012, 0.04217164, ..., 0.11136057, 0.38562581,
        1.        ]])

In [43]:

sorted(list(enumerate(similarity[0])), key=lambda x: x[1], reverse=True)[1:20]

[(276, 0.9733772868873092),
 (730, 0.5178110998810745),
 (81, 0.4926796496078711),
 (917, 0.4926796496078711),
 (252, 0.2580845280133163),
 (128, 0.2468808187192524),
 (360, 0.23664048105861918),
 (825, 0.21843925216946114),
 (629, 0.20867736353084287),
 (38, 0.18257068622788095),
 (245, 0.1791750999057027),
 (114, 0.17258359581182642),
 (254, 0.16795112809252516),
 (298, 0.1446572732370542),
 (59, 0.14241842988596462),
 (940, 0.14091906233860918),
 (195, 0.13679073087097762),
 (284, 0.13617556112417506),
 (965, 0.13579317164344284)]

In [44]:

def recommend(title):

        indx = job_df[job_df['Title'] == title].index[0]
        indx = job_df.index.get_loc(indx)
        distances = sorted(list(enumerate(similarity[indx])), key=lambda x: x[1], reverse=True)[1:20]

        jobs = []
        for i in distances:
            jobs.append(job_df.iloc[i[0]].Title)
        return jobs

In [45]:
recommend('site director knowledg univers')

['site director knowledg univers educ llc',
 'teacher knowledg univers',
 'assist teacher knowledg univers',
 'assist teacher knowledg univers',
 'cook knowledg univers',
 'immedi open for assist teacher la petit academi',
 'summer school age assist children courtyard',
 'hvac instructor vatterott educ center',
 'temporari coordin site oper macyscom maci',
 'fellowship program coordin connecticut children medic center',
 'medic offic assist instructor concord career colleg inc',
 'youth camp residenti assist activ coordin new york ny el educ servic',
 'pharmaci adjunct instructor brown macki colleg',
 'secur offic regular securita usa',
 'secur offic 100000 job coalit securita usa',
 'after school day camp counselor ii ymca of greenvil',
 'faculti i on call zenith educ group',
 'faculti it it support specialist zenith educ group',
 'handbag sell specialist part time bloomingdal chevi chase md bloomingdal']

In [46]:
import pickle
pickle.dump(job_df,open('df.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [47]:
import pandas as pd
print(pd.__version__)

2.2.2
